# Change-Aware Fuzzing Test Generation - Interactive Testing

This notebook allows you to interactively test the fuzzing pipeline components:
1. **Patch Analysis** - Extract changed functions and lines from diffs
2. **Test Generation** - Generate Hypothesis property-based tests
3. **Coverage Analysis** - Analyze coverage of changed lines only

Use this to verify that the fuzzing test generation works correctly before integrating with SWE-bench.

In [1]:
# Setup
import sys
from pathlib import Path

# Add project root to path
PROJECT_ROOT = Path.cwd()
sys.path.append(str(PROJECT_ROOT))

from verifier.dynamic_analyzers.patch_analyzer import PatchAnalyzer
from verifier.dynamic_analyzers.test_generator import HypothesisTestGenerator
from verifier.dynamic_analyzers.coverage_analyzer import CoverageAnalyzer

print("✓ Imports successful")

✓ Imports successful


## Test 1: Simple Division by Zero Fix

This is a classic example - adding a check for division by zero.

In [2]:
# Define a simple patch
patch_diff = """
--- a/math_utils.py
+++ b/math_utils.py
@@ -1,5 +1,7 @@
 def divide(a, b):
-    return a / b
+    if b == 0:
+        raise ValueError("Cannot divide by zero")
+    return a / b
"""

patched_code = """
def divide(a, b):
    if b == 0:
        raise ValueError("Cannot divide by zero")
    return a / b
"""

print("Patch:")
print(patch_diff)

Patch:

--- a/math_utils.py
+++ b/math_utils.py
@@ -1,5 +1,7 @@
 def divide(a, b):
-    return a / b
+    if b == 0:
+        raise ValueError("Cannot divide by zero")
+    return a / b



### Step 1: Analyze the Patch

In [3]:
# Analyze patch
analyzer = PatchAnalyzer()
patch_analysis = analyzer.parse_patch(patch_diff, patched_code)

print("\n" + "="*80)
print("PATCH ANALYSIS RESULTS")
print("="*80)
print(f"Changed functions: {patch_analysis.changed_functions}")
print(f"All changed lines: {patch_analysis.all_changed_lines}")
print(f"Changed lines by function: {patch_analysis.changed_lines}")
print(f"\nChange types:")
for change_type, changes in patch_analysis.change_types.items():
    if changes:
        print(f"  - {change_type}: {len(changes)} changes")
        for change in changes:
            print(f"      Line {change['line']}: {change['type']}")


PATCH ANALYSIS RESULTS
Changed functions: ['divide']
All changed lines: [2, 3, 4]
Changed lines by function: {'divide': [2, 3, 4]}

Change types:
  - conditionals: 1 changes
      Line 3: if_statement
  - exceptions: 1 changes
      Line 4: exception
  - operations: 1 changes
      Line 3: operation


### Step 2: Generate Fuzzing Tests

In [4]:
# Generate tests
generator = HypothesisTestGenerator()
test_code = generator.generate_tests(patch_analysis, patched_code)

print("\n" + "="*80)
print("GENERATED FUZZING TESTS")
print("="*80)
print(f"Total test functions: {test_code.count('def test_')}")
print(f"Total code length: {len(test_code)} characters")
print("\n" + "-"*80)
print("TEST CODE:")
print("-"*80)
print(test_code)
print("-"*80)


GENERATED FUZZING TESTS
Total test functions: 3
Total code length: 2025 characters

--------------------------------------------------------------------------------
TEST CODE:
--------------------------------------------------------------------------------
# Auto-generated tests for patch validation
import pytest
from hypothesis import given, strategies as st, settings
from hypothesis import assume
import sys
from pathlib import Path

# Import the module under test
# This assumes the module is available in PYTHONPATH

@given(st.one_of(st.none(), st.integers(min_value=-100, max_value=100), st.text()), st.one_of(st.none(), st.integers(min_value=-100, max_value=100), st.text()))
@settings(max_examples=50, deadline=1000)
def test_divide_boundaries(arg0, arg1):
    """Test boundary conditions for divide"""
    try:
        # Try calling with various boundary values
        result = divide(arg0, arg1)
        # If function returns, it should be deterministic
        result2 = divide(arg0, a

## Test 2: More Complex Patch - Input Validation

Let's test with a more complex patch that adds multiple conditionals.

In [6]:
patch_diff_2 = """
--- a/utils.py
+++ b/utils.py
@@ -5,6 +5,10 @@ def process_data(items):
     Process a list of items.
+    if not items:
+        return []
+    if not isinstance(items, list):
+        raise TypeError("items must be a list")
     return [item * 2 for item in items]
"""

patched_code_2 = """
def process_data(items):
    '''Process a list of items.'''
    if not items:
        return []
    if not isinstance(items, list):
        raise TypeError("items must be a list")
    return [item * 2 for item in items]
"""

# Analyze
patch_analysis_2 = analyzer.parse_patch(patch_diff_2, patched_code_2)

print("Changed functions:", patch_analysis_2.changed_functions)
print("Changed lines:", patch_analysis_2.all_changed_lines)
print("\nChange types:")
for change_type, changes in patch_analysis_2.change_types.items():
    if changes:
        print(f"  {change_type}: {len(changes)} changes")

Changed functions: ['process_data']
Changed lines: [6, 7, 8, 9]

Change types:
  conditionals: 1 changes
  exceptions: 1 changes
  operations: 1 changes


In [7]:
# Generate tests for the second patch
test_code_2 = generator.generate_tests(patch_analysis_2, patched_code_2)

print("\n" + "="*80)
print("GENERATED TESTS FOR INPUT VALIDATION PATCH")
print("="*80)
print(test_code_2)
print("="*80)


GENERATED TESTS FOR INPUT VALIDATION PATCH
# Auto-generated tests for patch validation
import pytest
from hypothesis import given, strategies as st, settings
from hypothesis import assume
import sys
from pathlib import Path

# Import the module under test
# This assumes the module is available in PYTHONPATH

@given(st.one_of(st.none(), st.integers(), st.text()))
@settings(max_examples=50, deadline=1000)
def test_process_data_boundaries(arg0):
    """Test boundary conditions for process_data"""
    try:
        # Try calling with various boundary values
        result = process_data(arg0)
        # If function returns, it should be deterministic
        result2 = process_data(arg0)
        assert result == result2, 'Function should be deterministic'
    except (ValueError, TypeError, AttributeError, ZeroDivisionError, KeyError, IndexError):
        pass  # Expected for invalid inputs

def test_process_data_exceptions():
    """Test exception handling for process_data"""
    # Test with

## Test 3: Real SWE-bench Example

Now let's try with a real example from one of your repos.

In [8]:
# Let's check if we have any real patches in the repos_temp directory
import os

repos_temp = Path("repos_temp")
if repos_temp.exists():
    repos = [d for d in repos_temp.iterdir() if d.is_dir()]
    print(f"Found {len(repos)} repositories in repos_temp/")
    for repo in repos[:5]:  # Show first 5
        print(f"  - {repo.name}")
else:
    print("No repos_temp directory found")

Found 1 repositories in repos_temp/
  - pytest-dev__pytest


## Test 4: Coverage Analysis (Mock)

Let's test the coverage analyzer with mock coverage data.

In [9]:
# Mock coverage data (simulating what coverage.py would return)
mock_coverage_data = {
    'files': {
        'math_utils.py': {
            'executed_lines': [1, 2, 3, 4],  # Lines that were executed
            'missing_lines': [],  # Lines that weren't executed
            'summary': {
                'covered_lines': 4,
                'num_statements': 4,
                'percent_covered': 100.0
            }
        }
    }
}

# Changed lines from our first example (divide function)
changed_lines = {
    'divide': [2, 3, 4]  # The if statement and raise statement
}

# Analyze coverage
coverage_analyzer = CoverageAnalyzer()
coverage_result = coverage_analyzer.calculate_changed_line_coverage(
    mock_coverage_data,
    changed_lines,
    all_changed_lines=[2, 3, 4]
)

print("\n" + "="*80)
print("COVERAGE ANALYSIS (Change-Aware)")
print("="*80)
print(f"Overall coverage: {coverage_result['overall_coverage']:.1%}")
print(f"Total changed lines: {coverage_result['total_changed_lines']}")
print(f"Covered changed lines: {coverage_result['total_covered_lines']}")
print(f"Covered lines: {coverage_result['covered_lines']}")
print(f"Uncovered lines: {coverage_result['uncovered_lines']}")
print(f"\nPer-function coverage:")
for func, cov in coverage_result['per_function_coverage'].items():
    print(f"  {func}: {cov:.1%}")


COVERAGE ANALYSIS (Change-Aware)
Overall coverage: 100.0%
Total changed lines: 3
Covered changed lines: 3
Covered lines: [2, 3, 4]
Uncovered lines: []

Per-function coverage:
  divide: 100.0%


## Test 5: Generate Coverage Report

In [10]:
# Generate a human-readable report
report = coverage_analyzer.generate_coverage_report(coverage_result, patch_analysis)
print(report)

CHANGE-AWARE COVERAGE REPORT

Changed Functions: divide
Total Changed Lines: 3
Covered Changed Lines: 3
Overall Coverage: 100.0%

Per-Function Coverage:
----------------------------------------
  ✓ divide: 100.0%

Covered Lines (3):
----------------------------------------
  [2, 3, 4]



## Test 6: Your Custom Patch

Use this cell to test with your own patches!

In [ ]:
# Paste your own patch here
custom_patch = """
--- a/yourfile.py
+++ b/yourfile.py
@@ -10,5 +10,6 @@
 def your_function():
+    # Your changes here
     pass
"""

custom_code = """
def your_function():
    # Your changes here
    pass
"""

# Analyze and generate tests
custom_analysis = analyzer.parse_patch(custom_patch, custom_code)
custom_tests = generator.generate_tests(custom_analysis, custom_code)

print("Custom patch analysis:")
print(f"  Changed functions: {custom_analysis.changed_functions}")
print(f"  Changed lines: {custom_analysis.all_changed_lines}")
print("\nGenerated tests:")
print(custom_tests)

## Summary and Next Steps

### What We've Tested:
1. ✅ Patch analysis extracts changed functions and lines correctly
2. ✅ Test generator creates Hypothesis property-based tests
3. ✅ Coverage analyzer focuses on changed lines only

### What's Working:
- The fuzzing pipeline components work independently
- Test generation targets changed code specifically
- Coverage analysis is change-aware

### What's Missing:
- Integration with `test_patch_singularity.py:run_evaluation()`
- Execution of generated tests in Singularity containers
- Combining fuzzing results with SWE-bench test results

### Next Steps:
1. Verify test execution works in Singularity (use `test_fuzzing_pipeline.py`)
2. Integrate fuzzing into the SWE-bench evaluation pipeline
3. Test on real SWE-bench instances

In [ ]:
# Utility: Save generated tests to a file for manual inspection
def save_tests(test_code, filename="generated_test.py"):
    with open(filename, 'w') as f:
        f.write(test_code)
    print(f"✓ Tests saved to {filename}")

# Example: save the first test
# save_tests(test_code, "test_divide_fuzzing.py")